# Introduction

This cell downloads the data as needed when not on Kaggle.

In [4]:
import os
from pathlib import Path

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
if iskaggle: path = Path('../input/titanic')
else:
    path = Path('titanic')
    if not path.exists():
        import zipfile,kaggle
        kaggle.api.competition_download_cli(str(path))
        zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 34.1k/34.1k [00:00<00:00, 8.45MB/s]

We'll be using numpy and pytorch for array calculations in this notebook, and pandas for working with tabular data, so we'll import them and set them to display using a bit more space than they default to.

In [7]:
import torch
import numpy as np
import pandas as pd
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)

# Cleaning the data